In [1]:
## Josh Blaz -- LOTR
import nltk
import re
import urllib.request
import lxml.html as lh
import io
import requests
import os
import glob


#NOTE: Sentiment140 Polarity values: 0: negative, 2: neutral, 4: positive


NOTE: Elvish text is translated awkwardly into the .txt format    
</br>
IE:   
</br>
►M MPR -F+MTRX MP ft PPtK P&RMPht: P. t. The last Two runes are the initials of Thror and Thrain.

In [200]:
# Used Chapterize to split books into chapters 
## https://github.com/JonathanReeve/chapterize
### The editions of the books I chose are slightly different than the editions chosen by LOTR project, the editions I 
### used have chapters that are merged together as sections of one another.

# These are lists containing strings of every chapter for each book
silm_chapters = []
hobbit_chapters = []
fellowship_chapters = []
twotowers_chapters = []
return_chapters = []

# Paths to directories storing book chapters
list_of_paths = ['/Users/blaz/Desktop/LOTR/silmarillion-chapters', '/Users/blaz/Desktop/LOTR/hobbit-chapters',\
                '/Users/blaz/Desktop/LOTR/fellowship-chapters', '/Users/blaz/Desktop/LOTR/twotowers-chapters',\
                '/Users/blaz/Desktop/LOTR/return-chapters']

for path in list_of_paths: # iterate through the list of folder paths for each book
    for file in sorted(glob.glob(os.path.join(path,'*.txt'))): # This gives us a sorted list of the files in each directory                                                         
        f = open(file, 'r') # open and read file               # allowing us to read in the chapters in order.
        txt = f.read()
    
        ## determine which path we're using and append it to the correct book chapter list
        if path == '/Users/blaz/Desktop/LOTR/silmarillion-chapters': 
            silm_chapters.append(txt)
        elif path == '/Users/blaz/Desktop/LOTR/hobbit-chapters':
            hobbit_chapters.append(txt)
        elif path == '/Users/blaz/Desktop/LOTR/fellowship-chapters': 
            fellowship_chapters.append(txt)
        elif path == '/Users/blaz/Desktop/LOTR/twotowers-chapters': 
            twotowers_chapters.append(txt)
        elif path == '/Users/blaz/Desktop/LOTR/return-chapters': 
            return_chapters.append(txt)

len(fellowship_chapters)

22

**Now, for each book, we have lists containing all of the chapters for that book.  
</br>
This was accomplished by iterating through the directories that store the chapters for each book using the "glob" and "os" modules.**

In [4]:
"""
Function that segments given chapter into a 2500 character segments to be sent to the API.

Params - chapter is a chapter of a book to be broken into segments

Returns a list of segments of the chapter.
"""
def Segmenter(chapter):
    segments = []
    
    # start and end indices for segmenting the text
    start = 0
    end = 2500
    
    while end < len(chapter) + 2500:
        segments.append(chapter[start:end])
        start = end
        end = end + 2500
    
    return segments #segments of input chapter

**This function allows us to split the chapters of each book into segments to send in our HTTP-Post JSON requests.   
</br>
I chose 2500 as the length because this is roughly the length in characters of a page (including spaces). **

In [161]:
#Lists of Lists of Lists storing all segments of all chapters for each book
# [[chapter1 segment 0-2500, chap1, segmenet 2500-5000]... [chapter2 segment0-2500, ...]...]
silm_segments = []
hobbit_segments = []
fellowship_segments = []
twotowers_segments = []
return_segments = []

# List containing the lists storing each books' chapters
list_of_books = [silm_chapters, hobbit_chapters, fellowship_chapters, twotowers_chapters, return_chapters]
# List allowing us to access the segment lists
list_of_segments = [silm_segments, hobbit_segments, fellowship_segments, twotowers_segments, return_segments]

# Access the lists above so that we can iteratively segment each chapter in the corpus
for i in range(len(list_of_books)):
    for book in list_of_books:
        for chapter in list_of_books[i]:
            list_of_segments[i].append(Segmenter(chapter))
            
#print(hobbit_segments[15]) ## chapter
print(hobbit_segments[0][0]) ## segments of chapter

found, however, that some single runes stand for two modern letters: th, ng, ee; other runes of the 
same kind ( ^ea and COst) were also sometimes used. The secret door was marked DM. From the side 
a hand pointed to this, and under it was written: rinMratwixN 

►M MPR -F+MTRX MP ft PPtK P&amp;RMPht: P. t. The last Two runes are the initials of Thror and Thrain. The 

StF+M *RI-m XRMB MPM + ► M HHIM MFKKH F + 

SMtTI * sn+ PIK ►MTFht-riXWT I W ft fc l + h M F &amp; 
moon-runes read by Elrond were: pi rr ini+m ntxp+ pm 

On the Map the compass points are marked in runes, with East at the top, as usual in dwarf-maps, 
and so read clockwise: E(ast), S(outh), W(est), N(orth). 




In [228]:
"""
Function that sends segments of 1 chapter through the Sentiment140 API.
In order to do so, it creates a JSON file and adds these segments to the file, then it sends an HTTP post to the 
API using the requests module.

Returns 

Note: Maximum of 700,000 characters per API request, though this shouldn't be a problem
"""

def Polarity(chapter_segments): # segments of a single chapter
    request = {'data':[]}
    polarityList = []
    counter = 0
    for segment in chapter_segments: # Fill JSON
        request['data'].append({'text':segment})
    r = requests.post('http://www.sentiment140.com/api/bulkClassifyJson?appid=blaz_j1@denison.edu', json=request)
    jso = r.json()
    for i in range(len(request['data'])-1):
        #print(jso['data'][i]['polarity'])
        polarityList.append(jso['data'][i]['polarity'])
        #print(jso['data'])
    
    polarityTotal = 0
    for value in polarityList:
        polarityTotal = polarityTotal + value
    
    polarityAVG = polarityTotal/len(polarityList)
    print(jso['data'])
    #print(len(polarityList))
    #print(polarityList)
    #print(polarityAVG)
    return polarityAVG

In [229]:
silm_polarity = []
hobbit_polarity = []
fellowship_polarity = []
twotowers_polarity = []
return_polarity = []

"""
for i in range(len(silm_segments)):
    for x in range(len(silm_segments[i])):
        if i == 0 and x ==0:
            #Polarity((silm_segments[i][x]))
            print((silm_segments[i][x]))
"""

for i in range(len(silm_segments)):
    if i == 20:
        print(Polarity(silm_segments[i]))

[{'text': "Of Turin Turambar \n\n\n\nRian, daughter of Belegund, was the wife of Huor, son \nof Galdor; and she was wedded to him two months before \nhe went with Hurin his brother to the Nirnaeth Arnoediad. \nWhen no tidings came of her lord she fled into the wild; but \nshe was aided by the Grey-elves of Mithrim, and when her \nson Tuor was born they fostered him. Then Rian departed \nfrom Hithlum, and going to the Haudh-en-Ndengin she laid \nherself down upon it and died. \n\nMorwen, daughter of Baragund, was the wife of Hurin, \nLord of Dor-lomin; and their son was Turin, who was born in \nthe year that Beren Erchamion came upon Luthien in the \nForest of Neldoreth. A daughter they had also who was \ncalled Lalaith, which is Laughter, and she was beloved by \nTurin her brother; but when she was three years old there \ncame a pestilence to Hithlum, borne on an evil wind out of \nAngband, and she died. \n\n\nNow after the Nimaeth Arnoediad Morwen abode still \nin Dor-lomin, for Turin

In [ ]:
# Example JSON request
d = {'data':[{'text': 'awesome'}, {'text': 'this fucking sucks idiot'}, {'text': 'dumby'}]}
d["data"].append({'text': 'okay'})
print(d)

#r = requests.post('http://www.sentiment140.com/api/bulkClassifyJson?appid=blaz_j1@denison.edu', json=d)
#print(r.status_code)
#r.json()

js = r.json()

# Example of accessing the polarity values
#print(js['data'])
js['data'][0]['polarity']
js['data'][1]['polarity']
js['data'][2]['polarity']
len(d['data'])